In [ ]:
import numpy as np, random, operator, pandas as pd, math

class Direction:
    def __init__(self, input_size, hidden_size, output_size):
        self.w1 = np.random.uniform(-1., 1., (input_size,hidden_size))
        self.w2 = np.random.uniform(-1., 1., (hidden_size,hidden_size))
        self.w3 = np.random.uniform(-1., 1., (hidden_size,output_size))
        
    def __repr__(self):
        return str(self.w1) + ',' + str(self.w2) + ',' + str(self.w3)
    
    #def __getitem__(self, key):
        #return getattr(self, key)
        #return self[key]
        
    def getWeights(self):
        return self.w1, self.w2, self.w3
    
class Fitness:
    def __init__(self, path):
        self.w1 = path[0]
        self.w2 = path[1]
        self.w3 = path[2]
        self.bias = 1
        self.layer1 = []
        self.layer2 = []
        
        self.distance = 0
        self.fitness = 0
        
        
    def sigmoid(self, x, derivative=False):
        return x*(1.0-x) if derivative else 1.0/(1.0+np.exp(-x))
    
    # calcular a ação a ser executada seguindo a maior probabilidade
    def feedforward(self, input=None):
        input_array = []

        if input is None:
            input_array = np.array(self.input, ndmin=2)
        else:
            input_array = np.array(input, ndmin=2)
        
        #print('input_array', input_array)
        #print('w1', self.w1)
        
        xxx = np.dot(input_array, self.w1)
        
        #print('xxx', xxx)
            
        self.layer1 = self.sigmoid(np.dot(input_array, self.w1) + self.bias)
        self.layer2 = self.sigmoid(np.dot(self.layer1, self.w2) + self.bias)
        self.output = self.sigmoid(np.dot(self.layer2, self.w3) + self.bias)
        
    def calDistance(self):
        
        #print('calcdistance', self)
        
        square.centerx, square.centery = WIDTH // 2, HEIGHT // 2
        
        collided = False
        
        while collided == False:
            
            distance_up = distance_down = distance_right = distance_left = 0
        
            for wall in walls:
                x1, x2 = square.centerx, square.centery
                y1, y2 = wall.centerx, wall.centery
                
                if wall.centerx == square.centerx:
                    if square.centery < wall.centery:
                        distance_up = math.hypot(x1-x2, y1-y2)
                        
                    if square.centery > wall.centery:
                        distance_down = math.hypot(x1-x2, y1-y2)
                        
                if wall.centery == square.centery:
                    if square.centerx < wall.centerx:
                        distance_right = math.hypot(x1-x2, y1-y2)
                        
                    if square.centerx > wall.centerx:
                        distance_left = math.hypot(x1-x2, y1-y2)
                    
            input_array = distance_right, distance_down, distance_left, distance_up
        
            self.feedforward(input_array)
        
            action = np.argmax(self.output)
            
            #print('action', action)

            screen.fill(BLACK)  

            dt = clock.tick(60)
            
            #0 direita
            #1 baixo
            #2 esquerda
            #3 cima
            
            if (action == 0):
                pos_x = velocity * dt
                square.move_ip(pos_x, 0)

            if (action == 1):
                pos_y = (velocity * dt)
                square.move_ip(0, pos_y)

            if (action == 2):
                pos_x = -(velocity * dt)
                square.move_ip(pos_x, 0)
                
            if (action == 3):
                pos_y = -(velocity * dt)
                square.move_ip(0, pos_y)

            for wall in walls:
                if square.colliderect(wall):
                    print('bateu')
                    collided = True

            if (collided == True):
                break;
            
            for wall in walls:
                pygame.draw.rect(screen, WHITE, wall)
                
            pygame.draw.rect(screen, GREEN, exit)

            pygame.draw.rect(screen, RED, square)
            
            
            #init_x, init_y = WIDTH // 2, HEIGHT // 2
            init_x, init_y = exit.centerx, exit.centery
            current_x, current_y = square.centerx, square.centery
                
            xDis = abs(current_x - init_x)
            #calcula a distancia dos eixos y
            yDis = abs(current_y - init_y)
            #eleva as diferenças ao quadrado, soma e calcula a raiz
            self.distance = np.sqrt((xDis ** 2) + (yDis ** 2))
            
            str_mov = 'distance = ' + str(round(self.distance, 2))
            text_mov = font.render(str_mov, True, WHITE)
            text_rect = text_mov.get_rect()
            text_rect.center = (WIDTH // 2, HEIGHT - 60)

            screen.blit(text_mov, text_rect)
            
            pygame.display.flip()

        return self.distance
        
    def getPathFitness(self):
        if (self.fitness == 0):
            distance = int(self.calDistance())
            if (distance > 0):
                self.fitness = 1 / distance
            self.fitness = 1
        return self.fitness

    def getDistance(self):
        return self.distance



import pygame
import sys

BLACK = pygame.Color(0, 0, 0)
WHITE = pygame.Color(255, 255, 255)
RED = pygame.Color(255, 0, 0)
GREEN = pygame.Color(0, 255, 0)
WIDTH, HEIGHT = 600, 300

SQUARE_WIDTH, SQUARE_HEIGHT = 5, 5
WALL_WIDTH, WALL_HEIGHT = 20, 20

velocity = 0.1

pygame.init()

screen = pygame.display.set_mode((WIDTH, HEIGHT))

clock = pygame.time.Clock()

square = pygame.Rect(WIDTH // 2, HEIGHT // 2, SQUARE_WIDTH, SQUARE_HEIGHT)


levels = [
    "WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW",
    "W           W E W            W",
    "W      WWWWWW   W            W",
    "W      W    W   W            W",
    "W      W    W   W            W",
    "W      W    W   W            W",
    "W      W        W            W",
    "W      W        W            W",
    "W      W    W   W            W",
    "W      W    W   W            W",
    "W      W    W   W            W",
    "W      W    W                W",
    "W      W    W                W",
    "W           W   W            W",
    "WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW"
]


left = 0
top = 0

walls = []
exit = 0

for row in levels:
    for col in row:
        if col == 'W':
            wall = pygame.Rect(left, top, WALL_WIDTH, WALL_HEIGHT)
            walls.append(wall)
        if col == 'E':
            exit = pygame.Rect(left, top, WALL_WIDTH, WALL_HEIGHT)
        left += WALL_WIDTH
    top += WALL_HEIGHT
    if left >= WIDTH:
        left = 0


font = pygame.font.Font('freesansbold.ttf', 12)
        
class GeneticAlgorithm:
    def __init__(self, populationSize, eliteSize, mutationRate, generations):
        #self.population = population
        self.populationSize = populationSize
        self.eliteSize = eliteSize
        self.mutationRate = mutationRate
        self.generations = generations
        self.currentPopulation = []
        self.bestFitnessMovimentations = 1
        self.bestDistance = 0

    def start(self):
        self.currentPopulation = self.initialPopulation(self.populationSize, 4, 2, 4)

        for i in range(0, self.generations):
            print('generation: ', i)
            self.currentPopulation = self.nextGeneration(self.currentPopulation, self.eliteSize, self.mutationRate)

        bestIndividualIndex = self.rankIndividuals(self.currentPopulation)[0][0]
        bestIndividual = self.currentPopulation[bestIndividualIndex]
        print(bestIndividual)
        
        
    def createPath(self, listDirections):
        return random.sample(listDirections, len(listDirections))


    def initialPopulation(self, populationSize, input_size, hidden_size, output_size):
        population = []

        #for i in range(0, populationSize):
            #population.append(self.createPath(listDirections))
            
        for i in range(0, populationSize):
            #population.append(Direction(4, 2, 4))
            individual = []
            individual.append(np.random.uniform(-1., 1., (input_size,hidden_size)))
            individual.append(np.random.uniform(-1., 1., (hidden_size,hidden_size)))
            individual.append(np.random.uniform(-1., 1., (hidden_size,output_size)))
            
            population.append(individual)
            
        print('population', population)
            
        return population


    def rankIndividuals(self, population):
        fitnessResults = {}
        
        for i in range(0, len(population)):
            #print('population', population[i])
            fitnessResults[i] = Fitness(population[i]).getPathFitness()
            
        return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = False)


    def selection(self, populationRanked, eliteSize):
        #print('populationRanked', populationRanked)

        selectionResults = []
        df = pd.DataFrame(np.array(populationRanked), columns=["Index", "Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100 * df.cum_sum / df.Fitness.sum()

        for i in range(0, eliteSize):
            selectionResults.append(populationRanked[i][0])

        for i in range(0, len(populationRanked) - eliteSize):
            pick = 100 * random.random()

            for i in range(0, len(populationRanked)):
                if pick <= df.iat[i, 3]:
                    selectionResults.append(populationRanked[i][0])
                    break
                    
        return selectionResults

    def matingPool(self, population, selectionResults):
        matingpool = []

        for i in range(0, len(selectionResults)):
            index = selectionResults[i]
            matingpool.append(population[index])
        return matingpool

    def breed(self, parent1, parent2):
        
        print('parent1', parent1)
        print('parent2', parent2)
        
        child = []
        
        
        # a reproducao vai ser diferente
        # obter cada array de pesos
        # mesclar as propriedades de cada array
        # adicionar ao filho cada array
        
        for i in range(len(parent1)):
            if i % 2 == 0:
                child.append(parent1[i])
            else:
                child.append(parent2[i])

        return child

    def breedPopulation(self, matingpool, eliteSize):
        children = []
        length = len(matingpool) - eliteSize
        pool = random.sample(matingpool, len(matingpool))

        for i in range(0, eliteSize):
            children.append(matingpool[i])

        for i in range(0, length):
            parent1 = pool[i]
            parent2 = pool[len(matingpool)-i-1]
            child = self.breed(parent1, parent2)
            children.append(child)

        return children


    def mutate(self, individual, mutationRate):
        for i in np.arange(3):
            #print('mutate weight', i)
            for neuron in range(len(individual[i])):
                if (random.random() < mutationRate):
                    newWeight = random.uniform(-1., 1)
                    individual[i][neuron] = newWeight

        return individual

    def mutatePopulation(self, population, mutationRate):
        mutatedPopulation = []

        for individual in range(0, len(population)):

            mutatedIndividual = self.mutate(population[individual], mutationRate)
            mutatedPopulation.append(mutatedIndividual)

        return mutatedPopulation

    def nextGeneration(self, currentGeneration, eliteSize, mutationRate):
        populationRanked = self.rankIndividuals(currentGeneration)
        selectionResults = self.selection(populationRanked, eliteSize)
        matingpool = self.matingPool(currentGeneration, selectionResults)
        children = self.breedPopulation(matingpool, eliteSize)
        return self.mutatePopulation(children, mutationRate) 


while True:

    events = pygame.event.get()
    
    for event in events:
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_SPACE:
          
                ga = GeneticAlgorithm(populationSize=10, eliteSize=2, mutationRate=0.01, generations=100)
                ga.start()


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
population [[array([[-0.74294917, -0.81884119],
       [-0.90959391,  0.32313104],
       [-0.69565461, -0.39981859],
       [-0.19442215, -0.20524283]]), array([[ 0.67969002, -0.97853893],
       [ 0.75386327,  0.21264931]]), array([[-0.33958677, -0.38176255,  0.61004216,  0.8131001 ],
       [-0.6222834 , -0.25883667, -0.07608704, -0.70506609]])], [array([[-0.9293126 ,  0.40253067],
       [ 0.90767594, -0.11185072],
       [-0.9311542 , -0.9535841 ],
       [ 0.76388599,  0.76237954]]), array([[ 0.33632171, -0.23239126],
       [ 0.17530769, -0.95011628]]), array([[-0.13055981,  0.08797685,  0.77660662, -0.09720493],
       [ 0.79119022, -0.06751014,  0.83010485,  0.62385552]])], [array([[ 0.16318809, -0.28652338],
       [-0.05457468,  0.55651554],
       [-0.05397776, -0.637827  ],
       [ 0.70104208, -0.88013796]]), array([[-0.345909  ,  0.82820715],
       [-0.82904661, -0.74679256]]), array([[

/home/joeltonguerreiro/workspace/env_python3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: overflow encountered in exp


bateu
bateu
bateu
bateu
bateu
